## Installation

First, we install the Dataflux Dataset for PyTorch.

In [ ]:
! pip install gcs-torch-dataflux

Then we need to install the rest of the rest of the expected packages for this demo.

In [ ]:
! pip install torch
! pip install lightning

## Preparation

After installing all required packages, we must perform some additional preparation.

First, we set up the [authentication](https://github.com/GoogleCloudPlatform/dataflux-pytorch?tab=readme-ov-file#configuration) needed to run the notebook.

Modify the `PROJECT_ID` field to your own project ID.

In [ ]:
from google.colab import auth
PROJECT_ID = "YOUR_PROJECT_ID"
auth.authenticate_user(project_id=PROJECT_ID)

## Lightning Checkpointing

The Dataflux PyTorch offers an optional implementation of PyTorch Lightning's checkpoints through implmentation of the CheckpointIO interface.

The methods that are supported are `save_checkpoint`, `load_checkpoint`, `remove_checkpoint` and `teardown`. 

First construct a DatafluxLightningCheckpoint.

Modify the `BUCKET_NAME` field to your own bucket name within the your project and `CKPT_PATH` field to the path you would like to test with.

In [ ]:
from dataflux_pytorch.lightning import DatafluxLightningCheckpoint

BUCKET_NAME="YOUR_BUCKET_NAME"
dataflux_ckpt = DatafluxLightningCheckpoint(project_name=PROJECT_ID, bucket_name=BUCKET_NAME)
CKPT_PATH = "checkpoints/epoch0.ckpt"

Ensure your datset, dataloader and model have been defined. This example pulls from the PyTorch demos datasets for simplicity.

In [ ]:
from lightning.pytorch.demos import WikiText2, LightningTransformer
from torch.utils.data import DataLoader

dataset = WikiText2()
dataloader = DataLoader(dataset, num_workers=1)

model = LightningTransformer(vocab_size=dataset.vocab_size)

In order to interact with Lightning Checkpoints you will need to define a trainer and run `trainer.fit()`. 

In [ ]:
from lightning import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint

trainer = Trainer(
    plugins=[dataflux_ckpt],
    min_epochs=4,
    max_epochs=5,
    max_steps=3,
    accelerator="cpu",
)
trainer.fit(model, dataloader)

Using the trainer you can now save, load and remove the checkpoint. 

For example you would save the checkpoint to the `CKPT_PATH` by calling `save_checkpoint`.

In [ ]:
trainer.save_checkpoint(CKPT_PATH)

You can verify the method by checking your bucket to see the checkpoint saved to `CKPT_PATH`.